In [1]:
# Import necessary modules
from tests.MOF_builder.functions.ciftemplate2graph import ct2g,node_vecs
import networkx as nx
import numpy as np
import os
import veloxchem as vlx
import re
from numpy.linalg import norm
from tests.MOF_builder.functions.bbcif_properties import bb2array,X_vecs,selected_type_vecs
from tests.MOF_builder.functions.place_bbs import superimpose,mag_superimpose

In [2]:
from src.MOF_builder.functions.frag_recognizer import process_linker_molecule
linker_file = 'ndi.xyz'
molecule = vlx.Molecule.read_xyz_file(linker_file)
linker_topic =2
center_frag_nodes_num,center_Xs,single_frag_nodes_num,frag_Xs= process_linker_molecule(molecule,linker_topic)

center is two points
[9, 14]
ditopic linker: center are two points
find connected X in edge:   24
find connected X in edge:   39
edges/diedge.cif is writen
center_frag: 46 [18, 33]


In [3]:
# Display edges of the graph


'''
Er1     Er     0.8416   0.75   0.1586
Er2     Er     0.6584   0.75   0.3414
Er3     Er     0.8416   0.25   0.8414
Er4     Er     0.6584   0.25   0.6586
V5     V     0.5   0.5   0.5
V6     V     0.5   0.0   0.5
Er7     Er     0.1584   0.25   0.8414
Er8     Er     0.3416   0.25   0.6586
Er9     Er     0.1584   0.75   0.1586
Er10     Er     0.3416   0.75   0.3414
V11     V     0.0   0.0   0.0
V12     V     0.0   0.5   0.0
loop_
_geom_bond_atom_site_label_1
_geom_bond_atom_site_label_2
_geom_bond_distance
_geom_bond_site_symmetry_2
_ccdc_geom_bond_type
Er1     Er2    10.0   .     S
Er1     V11    10.0   1_665     S
Er1     V12    10.0   1_655     S
Er2     V5    10.0   .     S
Er2     V6    10.0   1_565     S
Er3     Er4    10.0   .     S
Er3     V12    10.0   1_656     S
Er3     V11    10.0   1_656     S
Er4     V5    10.0   .     S
Er4     V6    10.0   .     S
V5     V6    10.0   .     S
V5     V6    10.0   1_565     S
V5     Er8    10.0   .     S
V5     Er10    10.0   .     S
V6     V5    10.0   1_545     S
V6     Er8    10.0   .     S
V6     Er2    10.0   1_545     S
V6     Er10    10.0   1_545     S
Er7     Er8    10.0   .     S
Er7     V11    10.0   1_556     S
Er7     V12    10.0   1_556     S
Er9     Er10    10.0   .     S
Er9     V11    10.0   1_565     S
Er9     V12    10.0   .     S
Er10     V6    10.0   1_565     S
V11     V12    10.0   1_545     S
V11     Er1    10.0   1_445     S
V11     V12    10.0   .     S
V11     Er3    10.0   1_454     S
V11     Er9    10.0   1_545     S
V11     Er7    10.0   1_554     S
V12     V11    10.0   1_565     S
V12     Er1    10.0   1_455     S
V12     Er3    10.0   1_454     S
V12     Er7    10.0   1_554     S
'''

'\nEr1     Er     0.8416   0.75   0.1586\nEr2     Er     0.6584   0.75   0.3414\nEr3     Er     0.8416   0.25   0.8414\nEr4     Er     0.6584   0.25   0.6586\nV5     V     0.5   0.5   0.5\nV6     V     0.5   0.0   0.5\nEr7     Er     0.1584   0.25   0.8414\nEr8     Er     0.3416   0.25   0.6586\nEr9     Er     0.1584   0.75   0.1586\nEr10     Er     0.3416   0.75   0.3414\nV11     V     0.0   0.0   0.0\nV12     V     0.0   0.5   0.0\nloop_\n_geom_bond_atom_site_label_1\n_geom_bond_atom_site_label_2\n_geom_bond_distance\n_geom_bond_site_symmetry_2\n_ccdc_geom_bond_type\nEr1     Er2    10.0   .     S\nEr1     V11    10.0   1_665     S\nEr1     V12    10.0   1_655     S\nEr2     V5    10.0   .     S\nEr2     V6    10.0   1_565     S\nEr3     Er4    10.0   .     S\nEr3     V12    10.0   1_656     S\nEr3     V11    10.0   1_656     S\nEr4     V5    10.0   .     S\nEr4     V6    10.0   .     S\nV5     V6    10.0   .     S\nV5     V6    10.0   1_565     S\nV5     Er8    10.0   .     S\nV5    

In [92]:
#pillar stack structure:
#regarding chain_node cif file:
    #find the pillar direction vector which is defined by the Al-Al vectors in chain_node cif file 
    #find the pair of x vectors, which are vertical to the pillar direction vector in chain_node cif file, could be more than one pairs
    #find the Al-Al length, read from the cif file
#regarding the linker cif file:
    #find the length of the linker which should be the maximum length of X vectors in the linker cif file

#regarding the template cif file:
    #find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file, 
    #find the edge vector in the template cif file, which should be Er-Er edge
    #find the length of the edge vector in the template cif file
    #fix one direction of the box, which should be parallel to the pilalar direction

#the unit_cell of the template cif file should be scaled to fit the length of the linker, and the box_vector which along pillar direction should be fixed
#when we want to place the node in the box, we should use a pair of x vectors which targets the connected V through the edge vector in the template cif file and the pillar direction vector, we use superimpose function to rotate and place the node at the right position
#when we want to place the edge in the box, we should consider all of the x vectors placed in box and find the connected connected x vectors in two sidde of the Er-Er edge, then we use superimpose function to rotate and place the edge at the right position 
from tests.MOF_builder.functions.bbcif_properties import bb2array,selected_type_vecs,calc_edge_len
from tests.MOF_builder.functions.place_bbs import superimpose

chain_node_cif = '1Alchain.cif'
template_cif = 'chain_rna.cif'
linker_cif = 'edges/diedge.cif'
#chain node
Al_vecs = selected_type_vecs(chain_node_cif,'.', 'Al',False)
chain_node_arr = bb2array(chain_node_cif,'.')

chain_node_arr = bb2array(chain_node_cif,'.')
metal_cvec = Al_vecs[0]-Al_vecs[1]
node_pillar_cvec = metal_cvec/norm(metal_cvec) 

x_vecs = selected_type_vecs(chain_node_cif,'.','X',False)


#find the x vectors which are vertical to the pillar direction
xx_pairs = []
for i in range(0,len(x_vecs)):
    for j in range(i+1,len(x_vecs)):
        xx_vec = x_vecs[i]-x_vecs[j]
        xx_vec = xx_vec/norm(xx_vec)
        if np.dot(xx_vec,metal_cvec) < 1e-3:
            xx_pairs.append((i,j))
print(xx_pairs)
#linker
linker_x_vecs = selected_type_vecs(linker_cif,'.','X',False)
#ditopic linker only has two x vectors
linker_length = calc_edge_len(linker_cif,'.') #length of the edge should be x-x length in linker cif file, unit angstrom
print(linker_length,'linker_length')

#template
gen = ct2g('chain_rna.cif', 'tests/templates')
net = next(gen)
TG, start, unit_cell, TVT, TET, TNAME, a, b, c, ang_alpha, ang_beta, ang_gamma, max_le, catenation = net
#ditopic linker should be connected to two V nodes directly 
#find path between two nodes which starts from V node to V node passing through Er nodes
Vnodes = [n for n in TG.nodes() if n.startswith('V')]
Ernodes = [n for n in TG.nodes() if n.startswith('Er')]

valid_paths = []
Vnodes = [n for n in TG.nodes() if n.startswith('V')]
for i in range(0, len(Vnodes)):
    for j in range(i+1, len(Vnodes)):
        source=Vnodes[i]
        target=Vnodes[j]
        path = nx.shortest_path(TG, source,target)
        if len(path) > 2:
            #if only two V nodes are there in the path, then it is a valid path
            if all([n.startswith('Er') for n in path[1:-1]]):
                #print(source, target, path)
                valid_paths.append(path)

NG = nx.Graph()
edges_info=TG.edges(data=True)
count = 1
le=0
pillars_cvecs = []
pillars_fvecs = []
edge_cvecs = []
edge_fvecs = []
for edge in edges_info:
    #node1,node2=edge[0],edge[1]
    edge_type=edge[2]['type']
    edge_label=edge[2]['label']
    edge_ccoord=edge[2]['ccoords']
    edge_fcoord=edge[2]['fcoords']
    edge_length=edge[2]['length']
    edge_index=edge[2]['index']
    edge_pd = edge[2]['pd']
    node1,node2 = edge_pd
 
    if edge_type[0]==edge_type[1]:
        #V-V find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file
        if edge_type[0]=='V':# chain of metal nodes
            #print(node1,node2,edge_type,edge_fcoord,edge_length,edge_index,edge_pd)
            name = node1+node2+'_'+str(count)
            pillars_cvecs.append(TG.nodes[node1]['ccoords']-TG.nodes[node2]['ccoords'])
            pillars_fvecs.append(TG.nodes[node1]['fcoords']-TG.nodes[node2]['fcoords'])
            NG.add_node(name, label = (node1,node2),type=edge_type, index=edge_index, ccoords=edge_ccoord, fcoords=edge_fcoord, cn=[], cifname=[])
            count+=1
        #Er-Er find the edge vector in the template cif file, which should be Er-Er edge
        else:
            le = max(edge_length,le) #find the length of the edge vector in the template cif file
            edge_cvecs.append(TG.nodes[node1]['ccoords']-TG.nodes[node2]['ccoords'])
            edge_fvecs.append(TG.nodes[node1]['fcoords']-TG.nodes[node2]['fcoords'])
            pass
            #edge direction 
            #print(node1,node2,edge_type,edge_fcoord,edge_length,edge_index,edge_pd)
            #get evecs

    else:
        #ignore Er-V
        #print(node1,node2,edge_type)
        pass

#in NG any VV node has 'V5' should be connected to antoher VV node with 'V11' and 'V12'
for path in valid_paths:
    source = path[0]
    target = path[-1]
    #in NG any VV node has source should be connected to antoher VV node with target
    vv_source_nodes = [node for node in NG.nodes() if source in NG.nodes[node]['label']]
    vv_target_nodes = [node for node in NG.nodes() if target in NG.nodes[node]['label']]
    #connect any source node to any target node


#find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file
count = 1
edges_cvecs=[]
edges_fvecs=[]
for vv_source_node in vv_source_nodes:
    for vv_target_node in vv_target_nodes:
        fcoord = (NG.nodes[vv_source_node]['fcoords'],NG.nodes[vv_target_node]['fcoords'])
        ccoord = (NG.nodes[vv_source_node]['ccoords'],NG.nodes[vv_target_node]['ccoords'])
        edges_cvecs.append(NG.nodes[vv_target_node]['ccoords']- NG.nodes[vv_source_node]['ccoords'])
        edges_fvecs.append(NG.nodes[vv_target_node]['fcoords']- NG.nodes[vv_source_node]['fcoords'])
        lbl = [0,0,0]
        NG.add_edge(vv_source_node,vv_target_node,key=(count,lbl[0],lbl[1],lbl[2]), label=lbl , length=le, fcoords=fcoord, ccoords=ccoord, index=count, pd=(vv_source_node,vv_target_node))
        count+=1
        


#fix one direction of the box, which should be parallel to the pilalar direction



#the unit_cell of the template cif file should be scaled to fit the length of the linker, and the box_vector which along pillar direction should be fixed


#find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file


#find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file


cns = []
for n in NG.nodes():
    cn = NG.degree(n) #Gets the degree (number of connections) of node n in the main graph G.
    ty = re.sub('[0-9]','',n) #Removes numeric characters from the node identifier to get its type.
    cns.append((cn, ty))
TVT = set(cns)



e_types = []
for e in NG.edges():
    e0 = e[0]
    e1 = e[1]
    l = sorted([re.sub('[^a-zA-Z]','',e0),re.sub('[^a-zA-Z]','',e1)]) #Sorts and strips non-alphabetic characters from node identifiers to determine the edge type.
    e_types.append((l[0],l[1]))
TET = set(e_types)


TVT = sorted(TVT, key=lambda x:x[0], reverse=True) # sort node with connected degree, the first one is the highest(full)-coordinated node
TET = sorted(TET, reverse=True) #sort node_pair by the node_index


[(0, 1), (2, 3)]
15.754259612561926 linker_length


In [95]:
#fix one direction of the box, which should be parallel to the pilalar direction

#rotate the node to pillar direction and put all nodes into the cartesian coordinate 

chain_node_coords = np.asarray([i[1] for i in chain_node_arr])


In [96]:
pillar_vecs =[]
for vec in pillars_cvecs:
    norm_vec = np.abs(vec/norm(vec))
    pillar_vecs.append(np.round(norm_vec,5))

if len(np.unique(pillar_vecs,axis=0)) >1:
    raise ValueError ('cannot find pillar vector in template net')
else:
    pillar_vec = np.unique(pillar_vecs,axis=0)
    pillar_vec = pillar_vec.flatten()
    print('pillar_vec in template:', pillar_vec)

beginning_point = [0.0,0.0,0.0]
pillar_vec,node_pillar_cvec
rms,rot,tran = superimpose([beginning_point,node_pillar_cvec],[beginning_point,pillar_vec])

pillar_oriented_nodexvec=np.dot(np.asarray(x_vecs),rot)
pillar_oriented_nodecoords = np.dot(chain_node_coords,rot)
nodexxxx = []
chain_node_positions = []
for n in NG.nodes():
    nodexxxx.append(NG.nodes[n]['ccoords']+pillar_oriented_nodexvec)
    chain_node_positions.append(NG.nodes[n]['ccoords']+pillar_oriented_nodecoords)




pillar_vec in template: [0. 1. 0.]


In [8]:
from tests.MOF_builder.functions.bbcif_properties import bb2array,X_vecs,selected_type_vecs
bb2array('1Alchain.cif', '.')
xvecs = X_vecs('1Alchain.cif', '.',False)
metal_vecs = selected_type_vecs('1Alchain.cif', '.','Al',False)

In [9]:
def assign_node_vecs2edges(nodes_dir,TG, unit_cell, SYM_TOL, template_name):


	'''
	The assign_node_vecs2edges function is designed to assign node vectors to edges in a graph TG, 
	using unit cell transformations and checking for deviations. 

	TG: A networkx graph object representing the target graph.
	unit_cell: The unit cell matrix used for transformation.
	SYM_TOL: Symmetry tolerance, not explicitly used in the function but possibly relevant for other parts of the process.
	template_name: The name of the template, used in warning messages.

	Return the dictionary containing edge assignments for each node.
	'''
	#Initialize Edge Assignment Dictionary
	edge_assign_dict = dict((k,{}) for k in TG.nodes())
	#Loop Through Nodes in the Graph
	##Loop through each node in the graph TG, extracting the node name and node dictionary.
	##cif: The name of the CIF file associated with the node.

	for n in TG.nodes(data=True):

		name,ndict = n
		cif = ndict['cifname']

		#bbxlabels: Labels of the building block vectors.
		#nodlabels: Labels of the node vectors.
		#bbxvec: Vectors of the building block.
		#nodvec: Node vectors after unit cell transformation.

		bbxlabels = np.array([l[0] for l in X_vecs(cif,nodes_dir, True)])
		nodlabels = np.array([l[0] for l in node_vecs(n[0], TG, unit_cell, True)])

		bbxvec = X_vecs(cif, nodes_dir, False)
		nodvec = node_vecs(n[0], TG, unit_cell, False)
		
		#Superimpose Vectors
		##mag_superimpose: Function to superimpose vectors, returning RMSD, rotation, and translation.
		##aff_b: Apply the rotation and translation to the building block vectors.
		##laff_b: Concatenate labels with transformed building block vectors.
		##lnodvec: Concatenate labels with node vectors.
		rmsd,rot,tran = mag_superimpose(bbxvec, nodvec)
		aff_b = np.dot(bbxvec,rot) + tran
		laff_b = np.c_[bbxlabels,aff_b]
		lnodvec = np.c_[nodlabels,nodvec]
		
		#Initialize Distance Matrix and Compute Distances
		#distance_matrix: Matrix to store distances between vectors.
		#For each pair of vectors, compute the normalized vectors 
		# and the Euclidean distance between them, storing in the distance matrix.
		asd = []
		asd_append = asd.append

		distance_matrix = np.zeros((len(laff_b),len(laff_b)))
		nrow = ncol = len(laff_b)
		
		for i in range(nrow):
			for j in range(ncol):
		
				v1 = laff_b[i]
				v1vec = np.array([float(q) for q in v1[1:]])
				v1vec /= norm(v1vec)
		
				v2 = lnodvec[j]
				v2vec = np.array([float(q) for q in v2[1:]])
				v2vec /= norm(v2vec)
		
				dist = np.linalg.norm(v1vec - v2vec)
				distance_matrix[i,j] += dist
		
		#Sort Distances and Assign Edges
		#distances: List of distances sorted by distance.
		#For each distance, extract the vectors and check if the edge is already used.
		#If not used, append the assignment details to asd and add the edge to used_edges.
		#Raise a warning if the distance exceeds a certain threshold (0.60).
		distances = []
		for i in range(nrow):
			for j in range(ncol):
				distances.append((distance_matrix[i,j],i,j))
		distances = sorted(distances, key=lambda x:x[0])
		
		used_edges = []
		
		for dist in distances:
		
			v1 = laff_b[dist[1]]
			v1vec = np.array([float(q) for q in v1[1:]])
			mag = np.linalg.norm(v1vec)
		
			v2 = lnodvec[dist[2]]
			ind = int(v2[0])
		
			edge_assign = ind
		
			if edge_assign not in used_edges:

				used_edges.append(edge_assign)
				asd_append([ind, v1[0], mag, v1vec, dist[0]])
				
				if dist[0] > 0.60:
					message = "There is a nodular building block vector that deviates from its assigned edge by more large\nthis may be fixed during scaling, but don't count on it!\n"
					message = message + "the deviation is for " + cif + " assigned to " + name + " for template " + template_name
					warnings.warn(message)
			
			if len(used_edges) == ncol:
				break

		elad = dict((k[0], (k[1],k[2],k[3])) for k in asd)
		edge_assign_dict[name] = elad
					
	return edge_assign_dict


In [ ]:
#test scale 
from tests.MOF_builder.functions.scale import scale
from tests.MOF_builder.functions.vertex_edge_assign import assign_node_vecs2edges
from tests.MOF_builder.functions.SBU_geometry import SBU_coords
from tests.MOF_builder.configuration import CONNECTION_SITE_BOND_LENGTH, SYMMETRY_TOL, FIX_UC, SCALING_ITERATIONS, PRE_SCALE, MIN_CELL_LENGTH, OPT_METHOD
ea_dict = assign_node_vecs2edges(nodes_dir,TG, unit_cell, SYMMETRY_TOL, template)
all_SBU_coords = SBU_coords(TG, ea_dict, CONNECTION_SITE_BOND_LENGTH)
sc_a, sc_b, sc_c, sc_alpha, sc_beta, sc_gamma, sc_covar, Bstar_inv, max_length, callbackresults, ncra, ncca, scaling_data = scale(all_SBU_coords,a,b,c,ang_alpha,ang_beta,ang_gamma,max_le,num_vertices,Bstar,alpha,num_edges,FIX_UC,SCALING_ITERATIONS,PRE_SCALE,MIN_CELL_LENGTH,OPT_METHOD)
				

NameError: name 'nodes_dir' is not defined

In [17]:
#test for scale 
import numpy as np
from scipy.optimize import minimize

def unit_cell_to_cartesian_matrix(aL,bL,cL,alpha,beta,gamma):
    pi = np.pi
    """Convert unit cell parameters to a Cartesian transformation matrix."""
    aL,bL,cL,alpha,beta,gamma = list(map(float, (aL,bL,cL,alpha,beta,gamma)))
    ax = aL
    ay = 0.0
    az = 0.0
    bx = bL * np.cos(gamma * pi / 180.0)
    by = bL * np.sin(gamma * pi / 180.0)
    bz = 0.0
    cx = cL * np.cos(beta * pi / 180.0)
    cy = (cL * bL * np.cos(alpha * pi /180.0) - bx * cx) / by
    cz = (cL ** 2.0 - cx ** 2.0 - cy ** 2.0) ** 0.5
    unit_cell = np.asarray([[ax,ay,az],[bx,by,bz],[cx,cy,cz]]).T
    return unit_cell

def fractional_to_cartesian(fractional_coords, T):
    """Convert fractional coordinates to Cartesian using the transformation matrix."""
    return np.dot(fractional_coords, T.T)

def cartesian_to_fractional(cartesian_coords, unit_cell_inv):
    """Convert Cartesian coordinates to fractional coordinates using the inverse transformation matrix."""
    return np.dot(cartesian_coords, unit_cell_inv.T)

def objective_function(params, old_cell_params, old_cartesian_coords, new_cartesian_coords):
    a_new, b_new, c_new, alpha_new, beta_new, gamma_new = params
    a_old, b_old, c_old, alpha_old, beta_old, gamma_old = old_cell_params

    # Compute transformation matrix for the old unit cell
    T_old = unit_cell_to_cartesian_matrix(a_old, b_old, c_old, alpha_old, beta_old, gamma_old)
    T_old_inv= np.linalg.inv(T_old)
    #backup
    old_fractional_coords = cartesian_to_fractional(old_cartesian_coords, T_old_inv)

    # Compute transformation matrix for the new unit cell
    T_new = unit_cell_to_cartesian_matrix(a_new, b_new, c_new, alpha_new, beta_new, gamma_new)
    T_new_inv= np.linalg.inv(T_new)

    # Convert the new Cartesian coordinates to fractional coordinate using the old unit cell
    fractional_coords_new = cartesian_to_fractional(new_cartesian_coords, T_new_inv)

    # Minimize the difference between the calculated new Cartesian coordinates and the provided new Cartesian coordinates
    return np.sum(np.linalg.norm(fractional_coords_new - old_fractional_coords, axis=1)**2)

# Example usage
# Old cell parameters (example values)
old_cell_params = [a, b, c, ang_alpha, ang_beta, ang_gamma]  # [a, b, c, alpha, beta, gamma]

# Old Cartesian coordinates of points (example values)
old_cartesian_coords = np.array([
    [0.0, 0.0, 0.0],    # Point 1
    [2.5, 3.0, 3.5],    # Point 2
    [1.25, 1.5, 5.25],  # Point 3
    [0.5, 2.4, 4.9]     # Point 4
])


# New Cartesian coordinates of the same points (example values)
new_cartesian_coords = np.array([
    [0.0, 0.0, 0.0],    # Point 1
    [5, 6, 7],    # Point 2
    [2.5, 3, 10.5],    # Point 3
    [1, 4.8, 9.8]     # Point 4
])

# Initial guess for new unit cell parameters (e.g., slightly modified cell)
initial_params = [4.5, 4.5, 4.5, 90.0, 90.0, 90.0]

# Bounds: a, b, c > 0; angles [0, 180]
bounds = [(0, None)] * 3 + [(20, 180)] * 3

# Optimize using L-BFGS-B to minimize the objective function
result = minimize(
    objective_function,
    x0=initial_params,
    args=(old_cell_params, old_cartesian_coords, new_cartesian_coords),
    method='L-BFGS-B',
    bounds=bounds
)

# Extract optimized parameters
optimized_params = np.round(result.x,5)
print("Optimized New Cell Parameters:", optimized_params)


Optimized New Cell Parameters: [63.86961 42.30462 77.11125 91.70537 93.78333 77.09491]


In [47]:
def apply_rotations_to_atom_positions(optimized_params, G, atom_positions):
    num_nodes = len(G.nodes())
    optimized_rotations = optimized_params.reshape(num_nodes, 3, 3)  # Reshape the optimized params into rotation matrices
    
    rotated_positions = {}
    
    for i, node in enumerate(G.nodes()):
        rotation_matrix = optimized_rotations[i]
        
        # Debugging: Check properties of the rotation matrix
        print(f"Node {node} - Rotation Matrix:\n{rotation_matrix}")
        
        # Check if the matrix is orthogonal: R^T R should equal the identity matrix
        orthogonality_check = np.dot(rotation_matrix.T, rotation_matrix)
        print(f"Node {node} - R^T R (should be identity):\n{orthogonality_check}")
        
        # Check determinant
        determinant = np.linalg.det(rotation_matrix)
        print(f"Node {node} - Determinant of Rotation Matrix: {determinant}")
        
        if not np.isclose(determinant, 1.0):
            print(f"Warning: Rotation matrix for Node {node} may not be a valid rotation (det = {determinant})")
        
        # Get the original atom positions for this node
        original_positions = atom_positions[node]
        
        # Calculate the center of mass (COM) for the node
        com = np.mean(original_positions, axis=0)  # Mean of all atoms' positions in this node
        print(f"Node {node} - COM: {com}")
        
        # Translate atom positions so that COM is at the origin
        translated_positions = original_positions - com
        print(f"Node {node} - Translated Positions (COM at origin):\n{translated_positions}")
        
        # Apply rotation around the COM (rotate the translated positions)
        rotated_translated_positions = np.dot(translated_positions, rotation_matrix.T)  # Rotate the translated positions
        print(f"Node {node} - Rotated Positions:\n{rotated_translated_positions}")
        
        # Translate back to the original position by adding COM
        rotated_positions[node] = rotated_translated_positions + com
        print(f"Node {node} - Final Rotated Positions:\n{rotated_positions[node]}")
        
    return rotated_positions


In [106]:
import numpy as np
from scipy.optimize import minimize, linear_sum_assignment
import networkx as nx

def find_optimal_pairings(node_i_positions, node_j_positions):
    """
    Find the optimal one-to-one pairing between atoms in two nodes using the Hungarian algorithm.
    """
    num_i, num_j = len(node_i_positions), len(node_j_positions)
    cost_matrix = np.zeros((num_i, num_j))
    for i in range(num_i):
        for j in range(num_j):
            cost_matrix[i, j] = np.linalg.norm(node_i_positions[i] - node_j_positions[j])

    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    print(list(zip(row_ind, col_ind)))
    return list(zip(row_ind, col_ind))

def find_edge_pairings(G, atom_positions):
    """
    Identify optimal pairings for each edge in the graph.

    Parameters:
        G (networkx.Graph): Graph structure with edges between nodes.
        atom_positions (dict): Positions of X atoms for each node.

    Returns:
        dict: Mapping of edges to optimal atom pairs.
              Example: {(0, 1): [(0, 3), (1, 2)], ...}
    """
    edge_pairings = {}

    for i, j in G.edges():
        node_i_positions = atom_positions[i]
        node_j_positions = atom_positions[j]

        # Find optimal pairings for this edge
        pairs = find_optimal_pairings(node_i_positions, node_j_positions)
        edge_pairings[(i, j)] = pairs


    return edge_pairings


def reorthogonalize_matrix(matrix):
    """
    Ensure the matrix is a valid rotation matrix with determinant = 1.
    """
    U, _, Vt = np.linalg.svd(matrix)
    R = np.dot(U, Vt)
    if np.linalg.det(R) < 0:
        U[:, -1] *= -1
        R = np.dot(U, Vt)
    return R

def compute_rotation_with_pairing(node_i_positions, node_j_positions, current_rotation_matrix):
    """
    Compute the optimal rotation matrix for node pairs, starting from the current rotation matrix.
    
    Parameters:
        node_i_positions (numpy.ndarray): Positions of X atoms in node i (Nx3 array).
        node_j_positions (numpy.ndarray): Positions of X atoms in node j (Mx3 array).
        current_rotation_matrix (numpy.ndarray): The current 3x3 rotation matrix for node i.

    Returns:
        rotation_matrix (numpy.ndarray): Optimized 3x3 rotation matrix for node i.
    """
    # Find optimal one-to-one pairings between the atoms
    pairs = find_optimal_pairings(node_i_positions, node_j_positions)
    
    # Extract paired positions
    paired_i = np.array([node_i_positions[i] for i, _ in pairs])
    paired_j = np.array([node_j_positions[j] for _, j in pairs])

    # Compute the centers of mass for both sets
    com_i = np.mean(paired_i, axis=0)
    com_j = np.mean(paired_j, axis=0)

    # Translate positions to the center of mass
    translated_i = paired_i - com_i
    translated_j = paired_j - com_j

    # Apply the current rotation matrix to node_i's positions
    rotated_translated_i = np.dot(translated_i, current_rotation_matrix.T)

    # Compute covariance matrix and SVD
    H = np.dot(rotated_translated_i.T, translated_j)
    U, _, Vt = np.linalg.svd(H)
    R = np.dot(U, Vt)

    # Ensure the resulting matrix is a valid rotation matrix (det = 1)
    if np.linalg.det(R) < 0:
        U[:, -1] *= -1
        R = np.dot(U, Vt)

    # Update the rotation matrix by combining the current and incremental rotation
    optimized_rotation_matrix = np.dot(R, current_rotation_matrix)

    return optimized_rotation_matrix




def objective_function(params, G, atom_positions, edge_pairings):
    """
    Objective function to minimize distances between paired atoms along edges.

    Parameters:
        params (numpy.ndarray): Flattened array of rotation matrices.
        G (networkx.Graph): Graph structure.
        atom_positions (dict): Original positions of X atoms for each node.
        edge_pairings (dict): Precomputed pairings for each edge.

    Returns:
        float: Total distance metric to minimize.
    """
    num_nodes = len(G.nodes())
    rotation_matrices = params.reshape(num_nodes, 3, 3)
    total_distance = 0.0

    for (i, j), pairs in edge_pairings.items():
        R_i = reorthogonalize_matrix(rotation_matrices[i])
        R_j = reorthogonalize_matrix(rotation_matrices[j])

        com_i = np.mean(atom_positions[i], axis=0)
        com_j = np.mean(atom_positions[j], axis=0)

        # Rotate positions around their mass center
        rotated_i_positions = np.dot(atom_positions[i] - com_i, R_i.T) + com_i
        rotated_j_positions = np.dot(atom_positions[j] - com_j, R_j.T) + com_j

        for idx_i, idx_j in pairs:
            dist = np.linalg.norm(rotated_i_positions[idx_i] - rotated_j_positions[idx_j])
            total_distance += dist ** 2

    return total_distance




def optimize_rotations(G, atom_positions):
    """
    Optimize rotations for all nodes in the graph.

    Parameters:
        G (networkx.Graph): Graph structure with edges between nodes.
        atom_positions (dict): Positions of X atoms for each node.

    Returns:
        list: Optimized rotation matrices for all nodes.
    """
    num_nodes = len(G.nodes())
    initial_rotations = np.tile(np.eye(3), (num_nodes, 1)).flatten()

    # Precompute edge-specific pairings
    edge_pairings = find_edge_pairings(G, atom_positions)

    result = minimize(
        objective_function,
        initial_rotations,
        args=(G, atom_positions, edge_pairings),
        method="L-BFGS-B",
        options={"maxiter": 1000, "disp": True}
    )

    

    optimized_rotations = result.x.reshape(num_nodes, 3, 3)
    optimized_rotations = [reorthogonalize_matrix(R) for R in optimized_rotations]
    
    # Print the optimized pairings after optimization
    print("Optimized Pairings (after optimization):")
    for (i, j), pairs in edge_pairings.items():
        print(f"Node {i} and Node {j}:")
        for idx_i, idx_j in pairs:
            print(f"  node{i}_{idx_i} -- node{j}_{idx_j}")
    print()

    return optimized_rotations

def apply_rotations_to_atom_positions(optimized_rotations, G, atom_positions):
    """
    Apply the optimized rotation matrices to the atom positions.

    Parameters:
        optimized_rotations (list): Optimized rotation matrices for each node.
        G (networkx.Graph): Graph structure.
        atom_positions (dict): Original positions of X atoms for each node.

    Returns:
        dict: Rotated positions for each node.
    """
    rotated_positions = {}

    for i, node in enumerate(G.nodes()):
        R = optimized_rotations[i]
        original_positions = atom_positions[node]
        com = np.mean(original_positions, axis=0)

        # Translate, rotate, and translate back to preserve the mass center
        translated_positions = original_positions - com
        rotated_translated_positions = np.dot(translated_positions, R.T)
        rotated_positions[node] = rotated_translated_positions + com

    return rotated_positions


def save_xyz(filename, rotated_positions):
    """
    Save the rotated positions to an XYZ file for visualization.
    """
    with open(filename, "w") as file:
        num_atoms = sum(len(positions) for positions in rotated_positions.values())
        file.write(f"{num_atoms}\n")
        file.write("Optimized structure\n")

        for node, positions in rotated_positions.items():
            for pos in positions:
                file.write(f"X {pos[0]:.4f} {pos[1]:.4f} {pos[2]:.4f}\n")

# Example Usage
# Define xxxxatom positions for each node
xxxx_positions_dict = {}
for i in range(len(nodexxxx)):
    xxxx_positions_dict[i]=nodexxxx[i]
# Defina all node positions 
chain_node_positions_dict = {}
for i in range(len(chain_node_positions)):
    chain_node_positions_dict[i]=chain_node_positions[i]



G = nx.Graph()
G.add_nodes_from(list(xxxx_positions_dict))
G.add_edges_from([(0, 2), (0, 3),(1, 2), (1, 3)])  # Define a graph with 3 nodes and edges



# Optimize rotations
optimized_rotations = optimize_rotations(G, xxxx_positions_dict)





# Apply rotations
rotated_positions = apply_rotations_to_atom_positions(optimized_rotations, G, chain_node_positions_dict)

# Save results to XYZ
save_xyz("optimized_structure.xyz", rotated_positions)


[(0, 0), (1, 1), (2, 2), (3, 3)]
[(0, 0), (1, 1), (2, 2), (3, 3)]
[(0, 0), (1, 1), (2, 2), (3, 3)]
[(0, 0), (1, 1), (2, 2), (3, 3)]
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           36     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.43425D+04    |proj g|=  5.45697D-04
Optimized Pairings (after optimization):
Node 0 and Node 2:
  node0_0 -- node2_0
  node0_1 -- node2_1
  node0_2 -- node2_2
  node0_3 -- node2_3
Node 0 and Node 3:
  node0_0 -- node3_0
  node0_1 -- node3_1
  node0_2 -- node3_2
  node0_3 -- node3_3
Node 1 and Node 2:
  node1_0 -- node2_0
  node1_1 -- node2_1
  node1_2 -- node2_2
  node1_3 -- node2_3
Node 1 and Node 3:
  node1_0 -- node3_0
  node1_1 -- node3_1
  node1_2 -- node3_2
  node1_3 -- node3_3


At iterate    1    f=  1.43425D+04    |proj g|=  5.45697D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of 

 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment


def find_edge_pairings(G, atom_positions):
    """
    Identify optimal pairings for each edge in the graph.

    Parameters:
        G (networkx.Graph): Graph structure with edges between nodes.
        atom_positions (dict): Positions of X atoms for each node.

    Returns:
        dict: Mapping of edges to optimal atom pairs.
              Example: {(0, 1): [(0, 3), (1, 2)], ...}
    """
    edge_pairings = {}

    for i, j in G.edges():
        node_i_positions = atom_positions[i]
        node_j_positions = atom_positions[j]

        # Find optimal pairings for this edge
        pairs = find_optimal_pairings(node_i_positions, node_j_positions)
        edge_pairings[(i, j)] = pairs


    return edge_pairings




def optimal_assignments_for_all_rounds(workers, rounds_of_companies):
    """
    Assign workers to companies for multiple rounds, ensuring optimal assignment in each round.

    Parameters:
        workers (np.ndarray): Positions of workers, shape (num_workers, 2).
        rounds_of_companies (list): List of company positions for each round,
                                    each element is an np.ndarray of shape (num_companies, 2).

    Returns:
        list: List of assignments for each round, with total costs.
    """
    all_assignments = []
    for round_idx, companies in enumerate(rounds_of_companies):
        # Compute the cost matrix (Euclidean distances)
        cost_matrix = np.linalg.norm(workers[:, np.newaxis, :] - companies[np.newaxis, :, :], axis=2)

        # Solve the assignment problem
        worker_indices, company_indices = linear_sum_assignment(cost_matrix)

        # Compute the total cost for this round
        total_cost = cost_matrix[worker_indices, company_indices].sum()

        # Store the results for this round
        assignments = [(worker, company) for worker, company in zip(worker_indices, company_indices)]
        all_assignments.append({
            "round": round_idx,
            "assignments": assignments,
            "total_cost": total_cost,
        })

    return all_assignments

# Example data
workers = np.array([[1, 2], [3, 4], [5, 6]])  # Fixed worker positions

# Multiple rounds of company positions
rounds_of_companies = [
    np.array([[2, 2], [4, 4], [6, 6]]),  # Round 1 companies
    np.array([[1, 3], [3, 5], [5, 7]]),  # Round 2 companies
    np.array([[2, 1], [4, 3], [6, 5]]),  # Round 3 companies
]

# Run the optimization
results = optimal_assignments_for_all_rounds(workers, rounds_of_companies)

# Output the results
for result in results:
    print(f"Round {result['round']}:")
    print(f"  Assignments: {result['assignments']}")
    print(f"  Total cost: {result['total_cost']:.2f}")
